In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

load_dotenv() 

API_KEY = os.getenv("YOUTUBE_API_KEY")
API_VERSION = 'v3'
youtube = build('youtube', API_VERSION, developerKey=API_KEY)

In [ ]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part='snippet, statistics',
        id=channel_id
    )
    response = request.execute()

    if response['items']:

        data = dict(channel_name=response['items'][0]['snippet']['title'],
                    total_subscribers=response['items'][0]['statistics']['subscriberCount'],
                    total_views=response['items'][0]['statistics']['viewCount'],
                    total_videos=response['items'][0]['statistics']['videoCount'],
        )

        return data
    else:
        return None 

In [ ]:
# Read CSV into dataframe 
df = pd.read_excel("youtube_data_colombia.xlsx")
df.head()

In [ ]:
# Extract channel IDs and remove potential duplicates
channel_ids = df['NAME'].str.split('@').str[-1].unique()
channel_ids

In [ ]:
# Inicializar una lista para almacenar las estadísticas de los canales
channel_stats = []

# Iterar sobre los ID de canales y obtener estadísticas para cada uno
for channel_id in channel_ids:
    try:
        # Intentar obtener las estadísticas del canal
        stats = get_channel_stats(youtube, channel_id)
        
        # Si las estadísticas son válidas, agregar a la lista
        if stats is not None:
            channel_stats.append(stats)
        else:
            # Si las estadísticas son None, agregar un valor por defecto
            channel_stats.append({
                'channel_name': 'Unknown',
                'total_subscribers': 0,
                'total_views': 0,
                'total_videos': 0
            })
    
    except KeyError as e:
        # Manejar el error si falta la clave 'items' en la respuesta
        print(f"Error con el ID {channel_id}: No se encontró la clave 'items'. {e}")
        # Agregar un valor por defecto en caso de error
        channel_stats.append({
            'channel_name': 'Unknown',
            'total_subscribers': 0,
            'total_views': 0,
            'total_videos': 0
        })
    
    except Exception as e:
        # Capturar otros errores generales
        print(f"Error con el ID {channel_id}: {e}")
        # Agregar un valor por defecto en caso de error
        channel_stats.append({
            'channel_name': 'Unknown',
            'total_subscribers': 0,
            'total_views': 0,
            'total_videos': 0
        })


In [ ]:
# Convert the list of stats to a df
stats_df = pd.DataFrame(channel_stats)

In [ ]:
df.reset_index(drop=True, inplace=True)
stats_df.reset_index(drop=True, inplace=True)


# Concatenate the dataframes horizontally
combined_df = pd.concat([df, stats_df], axis=1)

In [ ]:
# Drop the 'channel_name' column from stats_df (since 'NOMBRE' already exists)
# combined_df.drop('channel_name', axis=1, inplace=True)


# Save the merged dataframe back into a CSV file
combined_df.to_csv('updated_youtube_data_co.csv', index=False)


combined_df.head(10)